## 使用 `czsc` 进行选股
---


In [1]:
import sys
sys.path.insert(0, ".")
sys.path.insert(0, "..")

import czsc
print(czsc.__version__)

0.5.4


In [2]:
from datetime import datetime
from typing import List
import traceback
from tqdm import tqdm_notebook as tqdm
from czsc.analyze import KlineAnalyze

In [3]:
# 导入聚宽数据
from czsc.data.jq import *

# 如果是第一次使用需要设置 token
# set_token("手机号", "密码")


### 缠论日线笔中枢三买选股
---

条件描述： 最近五笔走势，前三笔构成中枢，第四笔离开中枢，第五笔向下不回中枢



In [4]:
def selector(symbols: List):
    """输入股票列表，输入符合买点定义的股票"""
    res = []
    for symbol in tqdm(symbols, desc="缠论日线笔中枢三买选股"):
        try:
            kline = get_kline(symbol=symbol, end_date=datetime.now(), freq="D", count=1000)
            ka = KlineAnalyze(kline, ma_params=(5, 34, 120, 233))
            points = ka.bi_list[-6:]
            
            if len(points) == 6 and points[-1]['fx_mark'] == "d":
                zs_g = min([x['bi'] for x in points[:4] if x['fx_mark'] == 'g'])
                zs_d = max([x['bi'] for x in points[:4] if x['fx_mark'] == 'd'])
                
                if points[-1]['bi'] > zs_g > zs_d:
                    res.append(symbol)
                    
        except:
            traceback.print_exc()
    return res

In [5]:
# 在创业板中选股
symbols = get_index_stocks("399006.XSHE")
selected = selector(symbols)

print("选股结果：", selected)


选股结果： ['300133.XSHE']
